In [10]:
from dotenv import load_dotenv
load_dotenv()
import os

In [11]:
from langfuse import Langfuse

langfuse = Langfuse(
  secret_key=os.getenv("LANGFUSE_SECRET_KEY"),
  public_key=os.getenv("LANGFUSE_PUBLIC_KEY"),
  host=os.getenv("LANGFUSE_HOST")
)

In [12]:
from llm_guard.input_scanners import Anonymize
from llm_guard.input_scanners.anonymize_helpers import BERT_LARGE_NER_CONF
from langfuse.openai import openai # OpenAI integration
from langfuse.decorators import observe, langfuse_context
from llm_guard.output_scanners import Deanonymize
from llm_guard.vault import Vault

In [13]:
vault = Vault()
 
@observe()
def anonymize(input: str):
  scanner = Anonymize(vault, preamble="Insert before prompt", allowed_names=["John Doe"], hidden_names=["Test LLC"],
                    recognizer_conf=BERT_LARGE_NER_CONF, language="en")
  sanitized_prompt, is_valid, risk_score = scanner.scan(prompt)
  return sanitized_prompt
 
@observe()
def deanonymize(sanitized_prompt: str, answer: str):
  scanner = Deanonymize(vault)
  sanitized_model_output, is_valid, risk_score = scanner.scan(sanitized_prompt, answer)
 
  return sanitized_model_output

In [14]:
@observe()
def summarize_transcript(prompt: str):
  sanitized_prompt = anonymize(prompt)
 
  answer = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        max_tokens=100,
        messages=[
          {"role": "system", "content": "Summarize the given court transcript."},
          {"role": "user", "content": sanitized_prompt}
        ],
    ).choices[0].message.content
 
  sanitized_model_output = deanonymize(sanitized_prompt, answer)
 
  return sanitized_model_output

In [15]:
prompt = "So, Ms. Hyman, you should feel free to turn your video on and commence your testimony. Ms. Hyman: Thank you, Your Honor. Good morning. Thank you for the opportunity to address this Committee. My name is Kelly Hyman and I am the founder and managing partner of the Hyman Law Firm, P.A. I’ve been licensed to practice law over 19 years, with the last 10 years focusing on representing plaintiffs in mass torts and class actions. I have represented clients in regards to class actions involving data breaches and privacy violations against some of the largest tech companies, including Facebook, Inc., and Google, LLC. Additionally, I have represented clients in mass tort litigation, hundreds of claimants in individual actions filed in federal court involving ransvaginal mesh and bladder slings. I speak to you"

@observe()
def main():
    return summarize_transcript(prompt)

main()

2024-12-11 12:31:49 [debug    ] No entity types provided, using default default_entities=['CREDIT_CARD', 'CRYPTO', 'EMAIL_ADDRESS', 'IBAN_CODE', 'IP_ADDRESS', 'PERSON', 'PHONE_NUMBER', 'US_SSN', 'US_BANK_NUMBER', 'CREDIT_CARD_RE', 'UUID', 'EMAIL_ADDRESS_RE', 'US_SSN_RE']


Some weights of the model checkpoint at dslim/bert-large-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2024-12-11 12:31:50 [debug    ] Initialized NER model          device=device(type='mps') model=Model(path='dslim/bert-large-NER', subfolder='', revision='13e784dccceca07aee7a7aab4ad487c605975423', onnx_path='dslim/bert-large-NER', onnx_revision='13e784dccceca07aee7a7aab4ad487c605975423', onnx_subfolder='onnx', onnx_filename='model.onnx', kwargs={}, pipeline_kwargs={'batch_size': 1, 'device': device(type='mps'), 'aggregation_strategy': 'simple', 'ignore_labels': ['O', 'CARDINAL']}, tokenizer_kwargs={'model_input_names': ['input_ids', 'attention_mask']})
2024-12-11 12:31:50 [debug    ] Loaded regex pattern           group_name=CREDIT_CARD_RE
2024-12-11 12:31:50 [debug    ] Loaded regex pattern           group_name=UUID
2024-12-11 12:31:50 [debug    ] Loaded regex pattern           group_name=EMAIL_ADDRESS_RE
2024-12-11 12:31:50 [debug    ] Loaded regex pattern           group_name=US_SSN_RE
2024-12-11 12:31:50 [debug    ] Loaded regex pattern           group_name=BTC_ADDRESS
2024-12-11 1

'Ms. Hyman, a lawyer with over 19 years of experience in mass torts and class actions, including cases against tech giants like Facebook and Google, is about to testify before a committee. She has represented clients in various legal actions, such as those related to data breaches, privacy violations, and mass tort litigation involving medical devices.'

In [ ]:
from langfuse.decorators import observe
from langfuse.openai import openai # OpenAI integration
 
@observe()
def story():
    return openai.chat.completions.create(
        model="gpt-3.5-turbo",
        max_tokens=100,
        messages=[
          {"role": "system", "content": "You are a great storyteller."},
          {"role": "user", "content": "Once upon a time in a galaxy far, far away..."}
        ],
    ).choices[0].message.content
 
@observe()
def main():
    return story()
 
main()